# Module 6 Challenge

In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gmaps
import requests
from citipy import citipy
from datetime import datetime
# Import the API keys.
from config import g_key
from config import weather_api_key

## Part 2

### Import the WeatherPy_database.csv file from Part 1 as a new DataFrame

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Port Elizabeth,ZA,-33.92,25.57,60.80,38,47,5.82,scattered clouds,0.0,0.0
1,1,Hilo,US,19.73,-155.09,78.80,61,75,4.72,broken clouds,0.0,0.0
2,2,Provideniya,RU,64.38,-173.30,56.82,45,17,5.39,few clouds,0.0,0.0
3,3,Arraial Do Cabo,BR,-22.97,-42.03,70.63,78,0,6.80,clear sky,0.0,0.0
4,4,Busselton,AU,-33.65,115.33,60.01,42,0,9.42,clear sky,0.0,0.0


### Filter the DataFrame

##### Get User Preferences

In [3]:
# Ask the customer to add a minimum and maximum temperature value, and rain/snow preferences.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
rain_pref = input("Do you want it to be raining? (yes/no) ")
snow_pref = input("Do you want it to be snowing? (yes/no) ")

Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


##### Generate New DataFrame from Preferences

In [5]:
if rain_pref == 'yes' and snow_pref == 'yes':
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) & 
                                           (city_data_df["Rain (inches)"] > 0.0) & 
                                           (city_data_df["Snow (inches)"] > 0.0)]

elif rain_pref == 'yes' and snow_pref == 'no':
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (inches)"] > 0.0) &
                                           (city_data_df["Snow (inches)"] == 0.0)]

elif rain_pref == 'no' and snow_pref == 'yes':
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) & 
                                           (city_data_df["Rain (inches)"] == 0.0) & 
                                           (city_data_df["Snow (inches)"] > 0.0)]
    
elif rain_pref == 'no' and snow_pref == 'no':
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) & 
                                           (city_data_df["Rain (inches)"] == 0.0) & 
                                           (city_data_df["Snow (inches)"] == 0.0)]

    
    
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
1,1,Hilo,US,19.73,-155.09,78.80,61,75,4.72,broken clouds,0.0,0.0
3,3,Arraial Do Cabo,BR,-22.97,-42.03,70.63,78,0,6.80,clear sky,0.0,0.0
11,11,Vaini,TO,-21.20,-175.20,75.20,78,75,8.05,broken clouds,0.0,0.0
13,13,Albany,US,42.60,-73.97,72.00,93,57,2.10,broken clouds,0.0,0.0
20,20,Republic,US,37.12,-93.48,70.00,88,75,5.82,broken clouds,0.0,0.0
24,24,Rikitea,PF,-23.12,-134.97,75.42,80,94,14.56,overcast clouds,0.0,0.0
28,28,La Macarena,CO,2.18,-73.78,71.11,95,63,2.35,broken clouds,0.0,0.0
29,29,Carnarvon,AU,-24.87,113.63,73.40,60,0,10.29,clear sky,0.0,0.0
33,33,Kapaa,US,22.08,-159.32,78.80,74,75,19.46,broken clouds,0.0,0.0
39,39,Ukwa,IN,21.97,80.47,79.77,73,99,4.85,overcast clouds,0.0,0.0


### Add the cities to a marker layer map with a pop-up marker for each city

In [6]:
# Information to include:
    # Hotel name
    # City
    # Country
    # Current weather description with the maximum temperature

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Current Description", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
1,Hilo,US,broken clouds,78.80,19.73,-155.09,
3,Arraial Do Cabo,BR,clear sky,70.63,-22.97,-42.03,
11,Vaini,TO,broken clouds,75.20,-21.20,-175.20,
13,Albany,US,broken clouds,72.00,42.60,-73.97,
20,Republic,US,broken clouds,70.00,37.12,-93.48,
24,Rikitea,PF,overcast clouds,75.42,-23.12,-134.97,
28,La Macarena,CO,broken clouds,71.11,2.18,-73.78,
29,Carnarvon,AU,clear sky,73.40,-24.87,113.63,
33,Kapaa,US,broken clouds,78.80,22.08,-159.32,
39,Ukwa,IN,overcast clouds,79.77,21.97,80.47,


In [8]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
1,Hilo,US,broken clouds,78.80,19.73,-155.09,Hilo Hawaiian Hotel
3,Arraial Do Cabo,BR,clear sky,70.63,-22.97,-42.03,Pousada Porto Praia
11,Vaini,TO,broken clouds,75.20,-21.20,-175.20,Keleti Beach Resort
13,Albany,US,broken clouds,72.00,42.60,-73.97,
20,Republic,US,broken clouds,70.00,37.12,-93.48,AmericInn by Wyndham Republic
24,Rikitea,PF,overcast clouds,75.42,-23.12,-134.97,Pension Maro'i
28,La Macarena,CO,broken clouds,71.11,2.18,-73.78,AGENCIA DE VIAJES Y TURISMO CRISTALES MACARENA
29,Carnarvon,AU,clear sky,73.40,-24.87,113.63,Hospitality Carnarvon
33,Kapaa,US,broken clouds,78.80,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
39,Ukwa,IN,overcast clouds,79.77,21.97,80.47,Chouksey Hotel & Rajat Transport


In [10]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

### Save and upload the new DataFrame as WeatherPy_vacation.csv

In [11]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

### Save and upload the new marker layer map as WeatherPy_vacation_map.png

In [12]:
# Downloaded from the interactive figure